# Predictions Overview

This is a small notebook to report and analyse Prediction Studio data on Predictions. The underlying data is from the Data-DM-Snapshot table that is used to populate the Prediction Studio screen with Prediction Performance, Lift, CTR etc.

As data this notebook accept data exported from PDC - which has a slightly altered format - as well as data directly exported from the pyGetSnapshot dataset in Pega.

For a description of the datamart tables see https://docs-previous.pega.com/decision-management/87/database-tables-monitoring-models.

Disclaimer: this is not a canned, robust and customer-facing notebook (yet). It's mostly used internally to validate Prediction data. Column names and file formats may need some more review to make it more robust.

In [ ]:
from pathlib import Path
import polars as pl
import json
from pdstools import readDSExport, Prediction

data_export = "<YOUR DATA FILE HERE>"

prediction = None
if data_export.endswith(".parquet"):
    # TODO - this should be similar to the JSON import and is by convention
    predictions_raw_data = pl.read_parquet(Path(data_export).expanduser())
    prediction = Prediction.from_pdc(predictions_raw_data.lazy())
elif data_export.endswith(".json"):
    # Assume this is an export from the PDC screen
    with open(Path(data_export).expanduser()) as f:
        predictions_raw_data = pl.from_dicts(json.loads(f.read())["pxResults"])
        prediction = Prediction.from_pdc(data_export)
elif data_export.endswith(".zip"):
    # Assuming a direct export from the dataset
    predictions_raw_data = readDSExport(data_export).collect()
    prediction = Prediction(predictions_raw_data.lazy())

predictions_raw_data.head().to_pandas().style

Quick glance at the available data. Selecting just the last day of the snapshots.

In [ ]:
from pdstools import Prediction, cdh_utils, errors
import datetime

cdh_utils.toPRPCDateTime(datetime.datetime.now())[0:15]

In [ ]:
prediction.summary_by_channel().collect().to_pandas().style

In [ ]:
prediction_summary_by_channel = (
    prediction.summary_by_channel(keep_trend_data=True)
    .with_columns(Prediction=pl.format("{} ({})", pl.col.Channel, pl.col.ModelName))
    .collect()
)

In [ ]:
import plotly.express as px

px.line(
    prediction_summary_by_channel.filter(pl.col("isMultiChannelPrediction").not_())
    .filter(pl.col("Channel") != "Unknown")
    .sort(["SnapshotTime"]),
    x="SnapshotTime",
    y="Performance",
    color="Prediction",
    title="Prediction Performance",
)

In [ ]:
px.line(
    prediction_summary_by_channel.filter(pl.col("isMultiChannelPrediction").not_())
    .filter(pl.col("Channel") != "Unknown")
    .sort(["SnapshotTime"]),
    x="SnapshotTime",
    y="Lift",
    color="Prediction",
    title="Prediction Lift",
).update_yaxes(tickformat=",.2%")

In [ ]:
px.line(
    prediction_summary_by_channel.filter(pl.col("isMultiChannelPrediction").not_())
    .filter(pl.col("Channel") != "Unknown")
    .sort(["SnapshotTime"]),
    x="SnapshotTime",
    y="CTR",
    facet_row="Prediction",
    color="Prediction",
    title="Prediction CTR",
).update_yaxes(tickformat=",.3%", matches=None).for_each_annotation(
    lambda a: a.update(text="")
)

In [ ]:
px.line(
    prediction_summary_by_channel.filter(pl.col("isMultiChannelPrediction").not_())
    .filter(pl.col("Channel") != "Unknown")
    .sort(["SnapshotTime"]),
    x="SnapshotTime",
    y="ResponseCount",
    facet_row="Prediction",
    color="Prediction",
    title="Prediction Responses",
).update_yaxes(matches=None).for_each_annotation(lambda a: a.update(text=""))